# Enhanced 1-Hour Trading Strategy Analysis

This notebook runs and analyzes the performance of our enhanced 1-hour trading strategy.

In [ ]:
import sys
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime, timedelta

# Add the project root to the Python path
sys.path.append('..')

# Import strategy and data loading utilities
from code.strategies.enhanced_1h_strategy import Strategy
from utilities.data_manager import DataManager

## Load and Prepare Data

We'll load 1-hour OHLCV data for our analysis.

In [ ]:
# Load data
symbol = 'BTCUSDT'
start_date = '2023-01-01'
end_date = '2023-12-31'

# Load data
data_manager = DataManager("binance")
df = data_manager.load("BTC/USDT", "1h", start_date="2023-01-01", end_date="2023-12-31")

print(f"Data shape: {df.shape}")
df.head()

## Initialize and Run Strategy

We'll initialize the strategy with optimized parameters for 1-hour trading.

In [ ]:
# Strategy parameters optimized for 1-hour timeframe
params = {
    'mode': 'both',  # Trade both long and short
    'max_open_positions': 3,
    'max_daily_loss_pct': 2.0,
    'max_position_size_pct': 5.0,
    'trailing_stop_pct': 1.0,
    'break_even_pct': 0.5,
    'max_trades_per_day': 15
}

# Initialize strategy
strategy = Strategy(params, df)

# Run backtest
initial_balance = 10000  # Starting with $10,000
leverage = 1  # No leverage for initial testing
open_fee_rate = 0.001  # 0.1% trading fee
close_fee_rate = 0.001  # 0.1% trading fee

strategy.run_backtest(
    initial_balance=initial_balance,
    leverage=leverage,
    open_fee_rate=open_fee_rate,
    close_fee_rate=close_fee_rate
)

## Performance Analysis

Let's analyze the strategy's performance with various metrics and visualizations.

In [ ]:
def calculate_performance_metrics(trades_df, equity_df, initial_balance):
    # Basic metrics
    total_trades = len(trades_df)
    winning_trades = len(trades_df[trades_df['net_pnl'] > 0])
    win_rate = winning_trades / total_trades if total_trades > 0 else 0
    
    # PnL metrics
    total_pnl = trades_df['net_pnl'].sum()
    avg_win = trades_df[trades_df['net_pnl'] > 0]['net_pnl'].mean() if winning_trades > 0 else 0
    avg_loss = trades_df[trades_df['net_pnl'] < 0]['net_pnl'].mean() if len(trades_df[trades_df['net_pnl'] < 0]) > 0 else 0
    profit_factor = abs(trades_df[trades_df['net_pnl'] > 0]['net_pnl'].sum() / 
                       trades_df[trades_df['net_pnl'] < 0]['net_pnl'].sum()) if len(trades_df[trades_df['net_pnl'] < 0]) > 0 else float('inf')
    
    # Risk metrics
    max_drawdown = (equity_df['equity'].cummax() - equity_df['equity']).max() / equity_df['equity'].cummax()
    sharpe_ratio = (equity_df['equity'].pct_change().mean() / equity_df['equity'].pct_change().std()) * np.sqrt(252)  # Annualized
    
    # Time-based metrics
    avg_trade_duration = (trades_df['close_time'] - trades_df['open_time']).mean()
    
    return {
        'Total Trades': total_trades,
        'Win Rate': f"{win_rate:.2%}",
        'Total PnL': f"${total_pnl:,.2f}",
        'Average Win': f"${avg_win:,.2f}",
        'Average Loss': f"${avg_loss:,.2f}",
        'Profit Factor': f"{profit_factor:.2f}",
        'Max Drawdown': f"{max_drawdown:.2%}",
        'Sharpe Ratio': f"{sharpe_ratio:.2f}",
        'Average Trade Duration': str(avg_trade_duration),
        'Final Balance': f"${strategy.final_equity:,.2f}",
        'Total Return': f"{(strategy.final_equity - initial_balance) / initial_balance:.2%}"
    }

# Calculate and display metrics
metrics = calculate_performance_metrics(strategy.trades_info, strategy.equity_record, initial_balance)
pd.DataFrame.from_dict(metrics, orient='index', columns=['Value'])

## Visualizations

Let's create some visualizations to better understand the strategy's performance.

In [ ]:
def plot_equity_curve(equity_df):
    plt.figure(figsize=(15, 7))
    plt.plot(equity_df.index, equity_df['equity'], label='Equity Curve')
    plt.title('Strategy Equity Curve')
    plt.xlabel('Date')
    plt.ylabel('Equity ($)')
    plt.grid(True)
    plt.legend()
    plt.show()

def plot_drawdown(equity_df):
    drawdown = (equity_df['equity'].cummax() - equity_df['equity']) / equity_df['equity'].cummax()
    plt.figure(figsize=(15, 7))
    plt.plot(equity_df.index, drawdown, label='Drawdown', color='red')
    plt.title('Strategy Drawdown')
    plt.xlabel('Date')
    plt.ylabel('Drawdown')
    plt.grid(True)
    plt.legend()
    plt.show()

def plot_trade_distribution(trades_df):
    plt.figure(figsize=(15, 7))
    sns.histplot(data=trades_df, x='net_pnl', bins=50)
    plt.title('Trade PnL Distribution')
    plt.xlabel('PnL ($)')
    plt.ylabel('Frequency')
    plt.grid(True)
    plt.show()

def plot_monthly_returns(equity_df):
    monthly_returns = equity_df['equity'].resample('M').last().pct_change()
    plt.figure(figsize=(15, 7))
    monthly_returns.plot(kind='bar')
    plt.title('Monthly Returns')
    plt.xlabel('Month')
    plt.ylabel('Return')
    plt.grid(True)
    plt.show()

# Create visualizations
plot_equity_curve(strategy.equity_record)
plot_drawdown(strategy.equity_record)
plot_trade_distribution(strategy.trades_info)
plot_monthly_returns(strategy.equity_record)

## Market Regime Analysis

Let's analyze how the strategy performs in different market regimes.

In [ ]:
def analyze_market_regimes(trades_df):
    regime_performance = trades_df.groupby('market_regime').agg({
        'net_pnl': ['count', 'sum', 'mean'],
        'duration': 'mean'
    }).round(2)
    
    regime_performance.columns = ['Number of Trades', 'Total PnL', 'Average PnL', 'Average Duration']
    return regime_performance

# Display market regime analysis
analyze_market_regimes(strategy.trades_info)

## Risk Management Analysis

Let's analyze how our risk management rules are performing.

In [ ]:
def analyze_risk_management(trades_df):
    # Analyze stop losses
    sl_trades = trades_df[trades_df['close_reason'].str.contains('SL')]
    sl_percentage = len(sl_trades) / len(trades_df) if len(trades_df) > 0 else 0
    
    # Analyze daily loss limits
    daily_pnl = trades_df.groupby(trades_df['close_time'].dt.date)['net_pnl'].sum()
    days_hit_loss_limit = len(daily_pnl[daily_pnl < -initial_balance * (params['max_daily_loss_pct'] / 100)])
    
    # Analyze position sizing
    avg_position_size = trades_df['position_size'].mean()
    max_position_size = trades_df['position_size'].max()
    
    return {
        'Stop Loss Percentage': f"{sl_percentage:.2%}",
        'Days Hit Loss Limit': days_hit_loss_limit,
        'Average Position Size': f"${avg_position_size:,.2f}",
        'Maximum Position Size': f"${max_position_size:,.2f}"
    }

# Display risk management analysis
pd.DataFrame.from_dict(analyze_risk_management(strategy.trades_info), orient='index', columns=['Value'])

## Save Results

Let's save the backtest results for future reference.

In [ ]:
# Save equity record and trades info
strategy.save_equity_record('enhanced_1h_strategy')
strategy.save_trades_info('enhanced_1h_strategy')

print("Results saved successfully!")